In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

from sklearn import *
from lightgbm import LGBMRegressor

/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into w

In [6]:
import pickle
with open('dataset.pkl', 'rb') as f:
    X_trn, y_trn, X_val, y_val, X_all, y_all, X_tst, val_id = pickle.load(f)

In [11]:
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer, mean_squared_error
import scipy.stats

In [8]:
cv = PredefinedSplit((val_id-1).values)

In [12]:
def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [ ]:
param_dist = {
    'random_state': [77],
    'num_leaves': [40, 50, 60, 70, 80, 90, 100], #7
    'n_estimators': [200, 300, 400, 500, 600], # 5
    'learning_rate': scipy.stats.uniform(0.03, 0.2),
    'min_child_samples': scipy.stats.randint(10, 50),
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'subsample': [0.7, 0.8, 0.9, 1.0],
}
rscv = RandomizedSearchCV(LGBMRegressor(), param_dist, n_iter=100, 
                          scoring=make_scorer(RMSLE,greater_is_better=False),
                         cv=cv, verbose=2)

In [15]:
rscv.fit(X_all, y_all)

Fitting 1 folds for each of 100 candidates, totalling 100 fits
[CV] colsample_bytree=0.7, learning_rate=0.12534346268112684, min_child_samples=21, n_estimators=400, num_leaves=80, random_state=77, subsample=1.0 
[CV]  colsample_bytree=0.7, learning_rate=0.12534346268112684, min_child_samples=21, n_estimators=400, num_leaves=80, random_state=77, subsample=1.0, total=  18.7s
[CV] colsample_bytree=0.8, learning_rate=0.13279280550440994, min_child_samples=16, n_estimators=700, num_leaves=60, random_state=77, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.2s remaining:    0.0s


[CV]  colsample_bytree=0.8, learning_rate=0.13279280550440994, min_child_samples=16, n_estimators=700, num_leaves=60, random_state=77, subsample=0.7, total=  25.4s
[CV] colsample_bytree=1.0, learning_rate=0.2041583194579822, min_child_samples=11, n_estimators=500, num_leaves=60, random_state=77, subsample=0.9 
[CV]  colsample_bytree=1.0, learning_rate=0.2041583194579822, min_child_samples=11, n_estimators=500, num_leaves=60, random_state=77, subsample=0.9, total=  18.8s
[CV] colsample_bytree=0.9, learning_rate=0.07851175334000232, min_child_samples=11, n_estimators=600, num_leaves=100, random_state=77, subsample=1.0 
[CV]  colsample_bytree=0.9, learning_rate=0.07851175334000232, min_child_samples=11, n_estimators=600, num_leaves=100, random_state=77, subsample=1.0, total=  28.4s
[CV] colsample_bytree=0.7, learning_rate=0.084645153447793, min_child_samples=20, n_estimators=500, num_leaves=80, random_state=77, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.084645153447793, mi

[CV]  colsample_bytree=0.8, learning_rate=0.19399447433482267, min_child_samples=49, n_estimators=400, num_leaves=60, random_state=77, subsample=0.8, total=  17.9s
[CV] colsample_bytree=1.0, learning_rate=0.055339382561021, min_child_samples=44, n_estimators=300, num_leaves=70, random_state=77, subsample=0.8 
[CV]  colsample_bytree=1.0, learning_rate=0.055339382561021, min_child_samples=44, n_estimators=300, num_leaves=70, random_state=77, subsample=0.8, total=  15.9s
[CV] colsample_bytree=1.0, learning_rate=0.1264119443004379, min_child_samples=33, n_estimators=600, num_leaves=50, random_state=77, subsample=0.8 
[CV]  colsample_bytree=1.0, learning_rate=0.1264119443004379, min_child_samples=33, n_estimators=600, num_leaves=50, random_state=77, subsample=0.8, total=  27.9s
[CV] colsample_bytree=1.0, learning_rate=0.16610875643889728, min_child_samples=47, n_estimators=600, num_leaves=100, random_state=77, subsample=0.9 
[CV]  colsample_bytree=1.0, learning_rate=0.16610875643889728, min

[CV]  colsample_bytree=0.7, learning_rate=0.12162382399110772, min_child_samples=12, n_estimators=800, num_leaves=60, random_state=77, subsample=0.9, total=  29.8s
[CV] colsample_bytree=1.0, learning_rate=0.10773889440672199, min_child_samples=24, n_estimators=200, num_leaves=90, random_state=77, subsample=0.7 
[CV]  colsample_bytree=1.0, learning_rate=0.10773889440672199, min_child_samples=24, n_estimators=200, num_leaves=90, random_state=77, subsample=0.7, total=  10.6s
[CV] colsample_bytree=0.9, learning_rate=0.039527900223209636, min_child_samples=21, n_estimators=800, num_leaves=80, random_state=77, subsample=0.8 
[CV]  colsample_bytree=0.9, learning_rate=0.039527900223209636, min_child_samples=21, n_estimators=800, num_leaves=80, random_state=77, subsample=0.8, total=  39.6s
[CV] colsample_bytree=0.9, learning_rate=0.0753633520425182, min_child_samples=43, n_estimators=200, num_leaves=50, random_state=77, subsample=0.7 
[CV]  colsample_bytree=0.9, learning_rate=0.0753633520425182

[CV]  colsample_bytree=0.9, learning_rate=0.14379595280440344, min_child_samples=25, n_estimators=800, num_leaves=100, random_state=77, subsample=0.9, total=  38.9s
[CV] colsample_bytree=0.7, learning_rate=0.18601032671888362, min_child_samples=45, n_estimators=700, num_leaves=90, random_state=77, subsample=1.0 
[CV]  colsample_bytree=0.7, learning_rate=0.18601032671888362, min_child_samples=45, n_estimators=700, num_leaves=90, random_state=77, subsample=1.0, total=  30.8s
[CV] colsample_bytree=0.8, learning_rate=0.05193627249280365, min_child_samples=43, n_estimators=500, num_leaves=80, random_state=77, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.05193627249280365, min_child_samples=43, n_estimators=500, num_leaves=80, random_state=77, subsample=0.7, total=  25.5s
[CV] colsample_bytree=0.9, learning_rate=0.03924409211004244, min_child_samples=20, n_estimators=700, num_leaves=70, random_state=77, subsample=0.7 
[CV]  colsample_bytree=0.9, learning_rate=0.0392440921100424

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 48.8min finished


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0], dtype=int64)),
          error_score='raise',
          estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'random_state': [77], 'num_leaves': [40, 50, 60, 70, 80, 90, 100], 'n_estimators': [200, 300, 400, 500, 600, 700, 800], 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10674ff98>, 'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10674f7b8>, 'colsample_bytree': [0.7, 0.8, 0.9, 1.0], 'subsample': [0.7, 0.8, 0.9, 1.0]},

In [18]:
rscv.best_params_

{'colsample_bytree': 1.0,
 'learning_rate': 0.04823859611776544,
 'min_child_samples': 24,
 'n_estimators': 500,
 'num_leaves': 40,
 'random_state': 77,
 'subsample': 0.8}

In [19]:
rscv.best_score_

-0.5083587649114817

In [96]:
# print('train', RMSLE(y_trn, lgb.predict(X_trn)))
# print('valid', RMSLE(y_val, lgb.predict(X_val)))

print('# params')
print(rscv.best_params_)
print()
print('# descriptors')
print(list(X_trn.columns))

# params
{'colsample_bytree': 0.8, 'learning_rate': 0.12101431676613092, 'min_child_samples': 17, 'n_estimators': 600, 'num_leaves': 70, 'random_state': 77, 'subsample': 1.0}

# descriptors
['dow', 'year', 'month', 'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors', 'median_visitors', 'max_visitors', 'count_observations', 'latitude', 'longitude', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'total_reserv_sum', 'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int', 'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id_target_mean', 'air_store_id_target_std', 'air_genre_name_target_mean', 'air_genre_name_target_std', 'air_area_name_target_mean', 'air_area_name_target_std']


In [97]:
test['visitors'] = np.expm1(lgb.predict(X_tst))
filename = '../output/180128_lgb5'
test[['id', 'visitors']].to_csv(filename+'.csv.gz', index=False, compression='gzip')